# Test for Initial Condition with Kida Vortex

In [66]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import jax_cfd.sb as cfd

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [67]:
# parameters

theta = 0 #np.pi/6
a = 2
b = 1
chi = a/b
c = np.sqrt(a**2 - b**2)

mu_0 = np.arctanh(b/a)

S = 1

L = 10
N = 64

In [ ]:
omega = -S/chi * (chi+1)/(chi-1)
omega

## Set up Cartesian Grids

In [69]:
x = np.linspace(-L/2,L/2,N,endpoint=True)
y = np.linspace(-L/2,L/2,N,endpoint=True)
X, Y = np.meshgrid(x, y)

In [70]:
# rotate coordinate system s.t. semi-major axis of vortex aligns with x-axis

theta_p = theta + np.pi/2
X_rot = np.cos(theta_p) * X + np.sin(theta_p) * Y
Y_rot = -np.sin(theta_p) * X + np.cos(theta_p) * Y

## Convert to Elliptical Grids

In [71]:
# solve quadratic equation to obtain p and q

p = (-(X_rot**2 + Y_rot**2 - c**2) + np.sqrt((X_rot**2 + Y_rot**2 - c**2)**2 + 4 * c**2 * Y_rot**2)) / (2 * c**2)
q = (-(X_rot**2 + Y_rot**2 - c**2) - np.sqrt((X_rot**2 + Y_rot**2 - c**2)**2 + 4 * c**2 * Y_rot**2)) / (2 * c**2)

nu_0 = np.arcsin( np.sqrt(p) )

nu = np.zeros_like(X_rot)

nu[np.where((X_rot >= 0) & (Y_rot >= 0))] = nu_0[np.where((X_rot >= 0) & (Y_rot >= 0))]
nu[np.where((X_rot < 0) & (Y_rot >= 0))] = np.pi - nu_0[np.where((X_rot < 0) & (Y_rot >= 0))]
nu[np.where((X_rot < 0) & (Y_rot < 0))] = np.pi + nu_0[np.where((X_rot < 0) & (Y_rot < 0))]
nu[np.where((X_rot >= 0) & (Y_rot < 0))] = 2 * np.pi - nu_0[np.where((X_rot >= 0) & (Y_rot < 0))]

In [72]:
mu = 1/2 * np.log(1 - 2*q + 2*np.sqrt(q**2 - q))

## Compute Analytical Solution of Streamfunction

In [73]:
psi_i = - (S * c**2) / (2 * (chi-1)) * (1/chi * np.cosh(mu)**2 * np.cos(nu)**2 + chi * np.sinh(mu)**2 * np.sin(nu)**2)
psi_o = - (S * c**2) / (4 * (chi-1)**2) * (1 + 2 * (mu - mu_0) + 2 * (chi-1)**2 * np.sinh(mu)**2 * np.sin(nu)**2 + (chi-1)/(chi+1) * np.exp(-2 * (mu - mu_0)) * np.cos(2 * nu))

In [74]:
inside_core = np.where(mu<=mu_0)
outside_core = np.where(mu>mu_0)

In [75]:
psi = np.zeros_like(psi_i)
psi[inside_core] = psi_i[inside_core]
psi[outside_core] = psi_o[outside_core]

In [ ]:
fig, ax = plt.subplots()
ax.contour(psi.transpose(), colors='black', linestyles=':')
ax.set_aspect('equal', 'box')
ax.set_title('Streamfunction')

## Solve Velocity Fields in Elliptical Grids

In [77]:
dpsi_i_dmu = - (S * c**2) / (2 * (chi - 1)) * np.sinh(2 * mu) * (1/chi * np.cos(nu)**2 + chi * np.sin(nu)**2)
dpsi_i_dnu = - (S * c**2) / (2 * (chi - 1)) * np.sin(2 * nu) * (-1/chi * np.cosh(mu)**2 + chi * np.sinh(mu)**2)

In [78]:
dpsi_o_dmu = - (S * c**2) / (4 * (chi - 1)**2) * (2 + 2 * (chi - 1)**2 * np.sinh(2 * mu) * np.sin(nu)**2 - 2 * (chi-1)/(chi+1) * np.exp(-2 * (mu - mu_0)) * np.cos(2 * nu))
dpsi_o_dnu = - (S * c**2) / (4 * (chi - 1)**2) * (2 * (chi - 1)**2 * np.sinh(mu)**2 * np.sin(2 * nu) - 2 * (chi-1)/(chi+1) * np.exp(-2 * (mu - mu_0)) * np.sin(2 * nu))

In [79]:
dpsi_dmu = np.zeros_like(mu)
dpsi_dmu[inside_core] = dpsi_i_dmu[inside_core]
dpsi_dmu[outside_core] = dpsi_o_dmu[outside_core]

dpsi_dnu = np.zeros_like(nu)
dpsi_dnu[inside_core] = dpsi_i_dnu[inside_core]
dpsi_dnu[outside_core] = dpsi_o_dnu[outside_core]

In [80]:
h = c * np.sqrt(np.sinh(mu)**2 * np.cos(nu)**2 + np.cosh(mu)**2 * np.sin(nu)**2)

In [81]:
UX_rot = -1/h**2 * (c * np.cosh(mu) * np.sin(nu) * dpsi_dmu + c * np.sinh(mu) * np.cos(nu) * dpsi_dnu)
UY_rot = 1/h**2 * (c * np.sinh(mu) * np.cos(nu) * dpsi_dmu - c * np.cosh(mu) * np.sin(nu) * dpsi_dnu)

## Rotate Back to Original Grids

In [82]:
UX = np.cos(theta_p) * UX_rot - np.sin(theta_p) * UY_rot
UY = np.sin(theta_p) * UX_rot + np.cos(theta_p) * UY_rot # adding background shear rate too

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,5))
UX_plot = ax[0].imshow(UX.transpose(), origin='lower')
UY_plot = ax[1].imshow(UY.transpose(), origin='lower')
ax[0].contour(psi.transpose(), colors='black', linestyles=':')
ax[1].contour(psi.transpose(), colors='black', linestyles=':')
ax[0].set_title('X Velocity')
ax[1].set_title('Y Velocity')
plt.colorbar(UX_plot, ax = ax[0])
plt.colorbar(UY_plot, ax = ax[1])

This is not quite accurate

In [84]:
# compute vorticity 

def central_diff(v, dx):
    """
    Compute the central difference of a 2 dimensional velocity field.
    """
    vx = v[0]
    vy = v[1]
    return (np.roll(vx, -1, axis=0) - np.roll(vx, 1, axis=0)) / (2 * dx), \
           (np.roll(vx, -1, axis=1) - np.roll(vx, 1, axis=1)) / (2 * dx), \
           (np.roll(vy, -1, axis=0) - np.roll(vy, 1, axis=0)) / (2 * dx), \
           (np.roll(vy, -1, axis=1) - np.roll(vy, 1, axis=1)) / (2 * dx)


def compute_vorticity(v, dx):
    """
    Compute the vorticity of a 2D velocity field.
    """

    dvx_dx, dvx_dy, dvy_dx, dvy_dy = central_diff(v, dx)
    return dvy_dx - dvx_dy

In [ ]:
U = np.array([UX, UY])

W = compute_vorticity(U, L/N)

fig, ax = plt.subplots()

W_plot = ax.imshow(W[1:-1, 1:-1].transpose(), origin='lower')
ax.contour(psi[1:-1, 1:-1].transpose(), colors='black', linestyles=':')
ax.set_title('Vorticity')

plt.colorbar(W_plot,)

In [ ]:
fig, ax = plt.subplots()

U_mag = np.sqrt(UX**2 + UY**2)
quiver_plot = ax.quiver(X, Y, UX, UY, U_mag, cmap='viridis')

ellipse = Ellipse(xy=(0.0, 0.0),
                width=2*b, height=2*a,
                angle=np.degrees(theta),
                facecolor='none', edgecolor='k',
                linestyle=':', linewidth=1.5,
                zorder=2)
ax.add_patch(ellipse)

ax.set_aspect('equal', 'box')
ax.set_title('Quiver Map')

plt.colorbar(quiver_plot)
plt.savefig(f'my_plot_for_kida_vortex_theta_{theta:.2f}.png', dpi=300)

## Compute Streamfunction on Cartesian Coordinates

In [87]:
def finite_difference(f, dx, axis):
    diff = np.roll(f, 1, axis) - np.roll(f, -1, axis)
    return diff / (2 * dx)

In [88]:
dpsi_dx = finite_difference(psi, L/N, 0)
dpsi_dy = finite_difference(psi, L/N, 1)

ux = -dpsi_dy #+ S * Y
uy = dpsi_dx

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,5))
ux_plot = ax[0].imshow(ux[1:-1, 1:-1].transpose(), origin='lower')
uy_plot = ax[1].imshow(uy[1:-1, 1:-1].transpose(), origin='lower')
ax[0].set_title('X Velocity')
ax[1].set_title('Y Velocity')
plt.colorbar(ux_plot, ax = ax[0])
plt.colorbar(uy_plot, ax = ax[1])

In [ ]:
u = np.array([ux, uy])

w = compute_vorticity(u, L/N)

fig, ax = plt.subplots()

w_plot = ax.imshow(w[2:-2, 2:-2].transpose(), origin='lower')
ax.set_title('Vorticity')
ax.contour(psi[2:-2, 2:-2].transpose(), colors='black', linestyles=':')

plt.colorbar(w_plot,)

In [91]:
ux_comp = np.cos(np.pi/2) * ux - np.sin(np.pi/2) * uy
uy_comp = np.sin(np.pi/2) * ux + np.cos(np.pi/2) * uy

ux_rot = np.rot90(ux_comp, k=3)
uy_rot = np.rot90(uy_comp, k=3)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,5))
ux_plot = ax[0].imshow(ux_rot[1:-1, 1:-1].transpose(), origin='lower')
uy_plot = ax[1].imshow(uy_rot[1:-1, 1:-1].transpose(), origin='lower')
ax[0].set_title('X Velocity')
ax[1].set_title('Y Velocity')
plt.colorbar(ux_plot, ax = ax[0])
plt.colorbar(uy_plot, ax = ax[1])

In [ ]:
u_rot = np.array([ux_rot, uy_rot])

w_rot = compute_vorticity(u_rot, L/N)

fig, ax = plt.subplots()

w_plot = ax.imshow(w_rot[2:-2, 2:-2].transpose(), origin='lower')
ax.set_title('Vorticity')
ax.contour(psi[2:-2, 2:-2], colors='black', linestyles=':')

plt.colorbar(w_plot,)

## Test on `jax_cfd`

In [94]:
domain = ((-np.pi, np.pi), (-np.pi, np.pi))
grid = cfd.grids.Grid((N, N), domain=domain)

In [103]:
def cartesian_to_elliptical(x, y, c):
  """
  Obtain corresponding elliptical coordinate values from Cartesian grid.

  Args:
    x_coord: Cartesian coordinate grid values

  Returns:
    Corresponding elliptical coordinate grid values
  """

  B = x**2 + y**2 - c**2

  p = (-B + np.sqrt(B**2 + 4 * c**2 * y**2)) / (2 * c**2)
  q = (-B - np.sqrt(B**2 + 4 * c**2 * y**2)) / (2 * c**2)

  # compute mu coordinates

  mu = 1/2 * np.log(1 - 2*q + 2*np.sqrt(q**2 - q))
  
  # compute nu coordinates

  print(p[31])
  nu_0 = np.arcsin( np.sqrt(p) )

  nu = np.zeros_like(x)

  nu[np.where((x >= 0) & (y >= 0))] = nu_0[np.where((x >= 0) & (y >= 0))]
  nu[np.where((x < 0) & (y >= 0))] = np.pi - nu_0[np.where((x < 0) & (y >= 0))]
  nu[np.where((x < 0) & (y < 0))] = np.pi + nu_0[np.where((x < 0) & (y < 0))]
  nu[np.where((x >= 0) & (y < 0))] = 2 * np.pi - nu_0[np.where((x >= 0) & (y < 0))]

  return (mu, nu)

In [104]:
def kida_streamfunction(mu, nu):

    psi = np.zeros_like(mu)

    psi_i = - (S * c**2) / (2 * (chi-1)) * (1/chi * np.cosh(mu)**2 * np.cos(nu)**2 + chi * np.sinh(mu)**2 * np.sin(nu)**2)
    psi_o = - (S * c**2) / (4 * (chi-1)**2) * (1 + 2 * (mu - mu_0) + 2 * (chi-1)**2 * np.sinh(mu)**2 * np.sin(nu)**2 + (chi-1)/(chi+1) * np.exp(-2 * (mu - mu_0)) * np.cos(2 * nu))
    
    inside_core = np.where(mu<=mu_0)
    outside_core = np.where(mu>mu_0)

    psi[inside_core] = psi_i[inside_core]
    psi[outside_core] = psi_o[outside_core]

    return psi

In [ ]:
offsets = grid.cell_faces
bc = cfd.boundaries.shearingbox_boundary_conditions(grid.ndim, S, 0.0, 'psi')

# compute x velocity
x = grid.mesh((0.5,0))[0]
y = grid.mesh((0.5,0))[1]
mu, nu = cartesian_to_elliptical(x,y,c)
psi = kida_streamfunction(mu, nu)
psi = cfd.grids.GridVariable(cfd.grids.GridArray(psi, (0.5,0), grid), bc)
ux = cfd.finite_differences.central_difference(psi, axis=1)

In [ ]:
# compute y velocity
x = grid.mesh((1,0.5))[0]
y = grid.mesh((1,0.5))[1]
mu, nu = cartesian_to_elliptical(x,y,c)
psi = kida_streamfunction(mu, nu)
psi = cfd.grids.GridVariable(cfd.grids.GridArray(psi, (1,0.5), grid), bc)
uy = -1 * cfd.finite_differences.central_difference(psi, axis=0)

In [ ]:
x

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,5))
ux_plot = ax[0].imshow(ux.data.transpose(), origin='lower')
uy_plot = ax[1].imshow(uy.data.transpose(), origin='lower')
ax[0].set_title('X Velocity')
ax[1].set_title('Y Velocity')
plt.colorbar(ux_plot, ax = ax[0])
plt.colorbar(uy_plot, ax = ax[1])

In [46]:
ux_rot = -uy.data
uy_rot = ux.data

ux_rot = np.rot90(ux_rot, k=3)
uy_rot = np.rot90(uy_rot, k=3)

In [47]:
v = tuple(
      [cfd.grids.GridVariable(cfd.grids.GridArray(ux_rot, offsets[0], grid), cfd.boundaries.shearingbox_boundary_conditions(grid.ndim, S, 0.0, 'vx')),
      cfd.grids.GridVariable(cfd.grids.GridArray(uy_rot, offsets[1], grid), cfd.boundaries.shearingbox_boundary_conditions(grid.ndim, S, 0.0, 'vy'))]
      )

In [ ]:
UX = v[0].data
UY = v[1].data

fig, ax = plt.subplots(1, 2, figsize=(10,5))
ux_plot = ax[0].imshow(UX.transpose(), origin='lower')
uy_plot = ax[1].imshow(UY.transpose(), origin='lower')
ax[0].set_title('X Velocity')
ax[1].set_title('Y Velocity')
plt.colorbar(ux_plot, ax = ax[0])
plt.colorbar(uy_plot, ax = ax[1])

In [ ]:
u_rot = np.array([ux_rot, uy_rot])

w_rot = compute_vorticity(u_rot, L/N)

fig, ax = plt.subplots()

w_plot = ax.imshow(w_rot[2:-2, 2:-2].transpose(), origin='lower')
ax.set_title('Vorticity')

plt.colorbar(w_plot,)
